In [46]:
import pandas as pd
from PIL import Image
import numpy as np
import os
import cv2
from tqdm import tqdm
from pathlib import Path

# Configure notebook parameters

In [47]:
size = 256
dataset_base_path = '../../datasets'
images_dir = f"{dataset_base_path}/zalando-woman-outfit-dataset-raw"
output_dir_base = f'{dataset_base_path}/imageBasedModel/zalando/woman_outfits'

In [48]:
df = pd.read_csv(
    f'{dataset_base_path}/zalando-woman-outfit-dataset-raw/woman_outfit_clothes_with_img_file_path_and_color.csv'
)
df

,Unnamed: 0.1,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id,clothingItemColor,image_path
0,0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,PADDED VEST - Weste - black,Puma,https://www.zalando.ch/puma-ess-trainingsjacke...,https://img01.ztat.net/article/spp-media-p1/f0...,5Bay-ErCSwa,black,woman_outfits\5Bay-ErCSwa\0.jpg
1,1,1,https://www.zalando.ch/outfits/5Bay-ErCSwa/,SLFMY PERFECT TEE BOX CUT - T-Shirt basic - br...,Selected Femme,https://www.zalando.ch/selected-femme-sfmy-t-s...,https://img01.ztat.net/article/spp-media-p1/3b...,5Bay-ErCSwa,NaN,woman_outfits\5Bay-ErCSwa\1.jpg
2,2,2,https://www.zalando.ch/outfits/5Bay-ErCSwa/,MOM A LINE SHORT - Jeans Shorts - flash black,Levi's®,https://www.zalando.ch/levisr-mom-a-line-short...,https://img01.ztat.net/article/spp-media-p1/eb...,5Bay-ErCSwa,flash black,woman_outfits\5Bay-ErCSwa\2.jpg
3,3,3,https://www.zalando.ch/outfits/5Bay-ErCSwa/,530 UNISEX - Sneaker low - white,New Balance,https://www.zalando.ch/new-balance-mr530-sneak...,https://img01.ztat.net/article/spp-media-p1/7f...,5Bay-ErCSwa,white,woman_outfits\5Bay-ErCSwa\3.jpg
4,4,4,https://www.zalando.ch/outfits/5Bay-ErCSwa/,Sonnenbrille - brown,Swarovski,https://www.zalando.ch/swarovski-sonnenbrille-...,https://img01.ztat.net/article/spp-media-p1/03...,5Bay-ErCSwa,brown,woman_outfits\5Bay-ErCSwa\4.jpg
...,...,...,...,...,...,...,...,...,...,...
24546,24546,24546,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,SLFEMMA LOAFER - Slipper - black,Selected Femme,https://www.zalando.ch/selected-femme-schnuere...,https://img01.ztat.net/article/spp-media-p1/c4...,FOmLRaT0Rnu,black,woman_outfits\FOmLRaT0Rnu\0.jpg
24547,24547,24547,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,TABBY SHOULDER - Handtasche - black,Coach,https://www.zalando.ch/coach-polished-pebble-s...,https://img01.ztat.net/article/spp-media-p1/89...,FOmLRaT0Rnu,black,woman_outfits\FOmLRaT0Rnu\1.jpg
24548,24548,24548,https://www.zalando.ch/outfits/IMi9cMhhTSm/,High Heel Pumps - black,Even&Odd,https://www.zalando.ch/evenandodd-pumps-black-...,https://img01.ztat.net/article/spp-media-p1/f2...,IMi9cMhhTSm,black,woman_outfits\IMi9cMhhTSm\0.jpg
24549,24549,24549,https://www.zalando.ch/outfits/tUljaAGSTG6/,BASIC HOODED JACKET - Daunenjacke - black,Tommy Jeans,https://www.zalando.ch/tommy-jeans-basic-hoode...,https://img01.ztat.net/article/spp-media-p1/0f...,tUljaAGSTG6,black,woman_outfits\tUljaAGSTG6\2.jpg


# Replace backslashes

In [49]:
df['image_path'] = df['image_path'].apply(lambda x: x.replace('\\', '/'))
df

,Unnamed: 0.1,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id,clothingItemColor,image_path
0,0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,PADDED VEST - Weste - black,Puma,https://www.zalando.ch/puma-ess-trainingsjacke...,https://img01.ztat.net/article/spp-media-p1/f0...,5Bay-ErCSwa,black,woman_outfits/5Bay-ErCSwa/0.jpg
1,1,1,https://www.zalando.ch/outfits/5Bay-ErCSwa/,SLFMY PERFECT TEE BOX CUT - T-Shirt basic - br...,Selected Femme,https://www.zalando.ch/selected-femme-sfmy-t-s...,https://img01.ztat.net/article/spp-media-p1/3b...,5Bay-ErCSwa,NaN,woman_outfits/5Bay-ErCSwa/1.jpg
2,2,2,https://www.zalando.ch/outfits/5Bay-ErCSwa/,MOM A LINE SHORT - Jeans Shorts - flash black,Levi's®,https://www.zalando.ch/levisr-mom-a-line-short...,https://img01.ztat.net/article/spp-media-p1/eb...,5Bay-ErCSwa,flash black,woman_outfits/5Bay-ErCSwa/2.jpg
3,3,3,https://www.zalando.ch/outfits/5Bay-ErCSwa/,530 UNISEX - Sneaker low - white,New Balance,https://www.zalando.ch/new-balance-mr530-sneak...,https://img01.ztat.net/article/spp-media-p1/7f...,5Bay-ErCSwa,white,woman_outfits/5Bay-ErCSwa/3.jpg
4,4,4,https://www.zalando.ch/outfits/5Bay-ErCSwa/,Sonnenbrille - brown,Swarovski,https://www.zalando.ch/swarovski-sonnenbrille-...,https://img01.ztat.net/article/spp-media-p1/03...,5Bay-ErCSwa,brown,woman_outfits/5Bay-ErCSwa/4.jpg
...,...,...,...,...,...,...,...,...,...,...
24546,24546,24546,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,SLFEMMA LOAFER - Slipper - black,Selected Femme,https://www.zalando.ch/selected-femme-schnuere...,https://img01.ztat.net/article/spp-media-p1/c4...,FOmLRaT0Rnu,black,woman_outfits/FOmLRaT0Rnu/0.jpg
24547,24547,24547,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,TABBY SHOULDER - Handtasche - black,Coach,https://www.zalando.ch/coach-polished-pebble-s...,https://img01.ztat.net/article/spp-media-p1/89...,FOmLRaT0Rnu,black,woman_outfits/FOmLRaT0Rnu/1.jpg
24548,24548,24548,https://www.zalando.ch/outfits/IMi9cMhhTSm/,High Heel Pumps - black,Even&Odd,https://www.zalando.ch/evenandodd-pumps-black-...,https://img01.ztat.net/article/spp-media-p1/f2...,IMi9cMhhTSm,black,woman_outfits/IMi9cMhhTSm/0.jpg
24549,24549,24549,https://www.zalando.ch/outfits/tUljaAGSTG6/,BASIC HOODED JACKET - Daunenjacke - black,Tommy Jeans,https://www.zalando.ch/tommy-jeans-basic-hoode...,https://img01.ztat.net/article/spp-media-p1/0f...,tUljaAGSTG6,black,woman_outfits/tUljaAGSTG6/2.jpg


# add item index column

In [50]:
import re

df['clothingItemIndex'] = df['image_path'].apply(lambda x: re.search(r'/(\d+)\.jpg$', x).group(1))
df

,Unnamed: 0.1,Unnamed: 0,outfit_url,clothingItemName,clothingItemBrand,clothingItemDetailUrl,clothingItemPreviewImageUrl,id,clothingItemColor,image_path,clothingItemIndex
0,0,0,https://www.zalando.ch/outfits/5Bay-ErCSwa/,PADDED VEST - Weste - black,Puma,https://www.zalando.ch/puma-ess-trainingsjacke...,https://img01.ztat.net/article/spp-media-p1/f0...,5Bay-ErCSwa,black,woman_outfits/5Bay-ErCSwa/0.jpg,0
1,1,1,https://www.zalando.ch/outfits/5Bay-ErCSwa/,SLFMY PERFECT TEE BOX CUT - T-Shirt basic - br...,Selected Femme,https://www.zalando.ch/selected-femme-sfmy-t-s...,https://img01.ztat.net/article/spp-media-p1/3b...,5Bay-ErCSwa,NaN,woman_outfits/5Bay-ErCSwa/1.jpg,1
2,2,2,https://www.zalando.ch/outfits/5Bay-ErCSwa/,MOM A LINE SHORT - Jeans Shorts - flash black,Levi's®,https://www.zalando.ch/levisr-mom-a-line-short...,https://img01.ztat.net/article/spp-media-p1/eb...,5Bay-ErCSwa,flash black,woman_outfits/5Bay-ErCSwa/2.jpg,2
3,3,3,https://www.zalando.ch/outfits/5Bay-ErCSwa/,530 UNISEX - Sneaker low - white,New Balance,https://www.zalando.ch/new-balance-mr530-sneak...,https://img01.ztat.net/article/spp-media-p1/7f...,5Bay-ErCSwa,white,woman_outfits/5Bay-ErCSwa/3.jpg,3
4,4,4,https://www.zalando.ch/outfits/5Bay-ErCSwa/,Sonnenbrille - brown,Swarovski,https://www.zalando.ch/swarovski-sonnenbrille-...,https://img01.ztat.net/article/spp-media-p1/03...,5Bay-ErCSwa,brown,woman_outfits/5Bay-ErCSwa/4.jpg,4
...,...,...,...,...,...,...,...,...,...,...,...
24546,24546,24546,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,SLFEMMA LOAFER - Slipper - black,Selected Femme,https://www.zalando.ch/selected-femme-schnuere...,https://img01.ztat.net/article/spp-media-p1/c4...,FOmLRaT0Rnu,black,woman_outfits/FOmLRaT0Rnu/0.jpg,0
24547,24547,24547,https://www.zalando.ch/outfits/FOmLRaT0Rnu/,TABBY SHOULDER - Handtasche - black,Coach,https://www.zalando.ch/coach-polished-pebble-s...,https://img01.ztat.net/article/spp-media-p1/89...,FOmLRaT0Rnu,black,woman_outfits/FOmLRaT0Rnu/1.jpg,1
24548,24548,24548,https://www.zalando.ch/outfits/IMi9cMhhTSm/,High Heel Pumps - black,Even&Odd,https://www.zalando.ch/evenandodd-pumps-black-...,https://img01.ztat.net/article/spp-media-p1/f2...,IMi9cMhhTSm,black,woman_outfits/IMi9cMhhTSm/0.jpg,0
24549,24549,24549,https://www.zalando.ch/outfits/tUljaAGSTG6/,BASIC HOODED JACKET - Daunenjacke - black,Tommy Jeans,https://www.zalando.ch/tommy-jeans-basic-hoode...,https://img01.ztat.net/article/spp-media-p1/0f...,tUljaAGSTG6,black,woman_outfits/tUljaAGSTG6/2.jpg,2


# Create output directory

In [51]:
path = Path(output_dir_base)
path.mkdir(parents=True, exist_ok=True)

# Get max width and max height of all images

In [52]:
from PIL import UnidentifiedImageError

"""
get the pixel values from biggest images in data
maximum width and maximum height
"""

max_w = 0
max_w_img_path = ""
max_h = 0
max_h_img_path = ""

image_paths_with_errors = []

for root, dirs, files in tqdm(os.walk(images_dir)):
    for file in files:
        if file.endswith('.jpg'):
            image_path = os.path.join(root, file)

            try:
                with Image.open(image_path) as image:
                    image_w, image_h = image.size

                    if image_w > max_w:
                        max_w = image_w
                        max_w_img_path = image_path

                    if image_h > max_h:
                        max_h = image_h
                        max_h_img_path = image_path
            except UnidentifiedImageError:
                image_paths_with_errors.append(image_path)

print("Max width:", max_w, "image_path:", max_w_img_path)
print("Max height:", max_h, "image_path:", max_h_img_path)
print(f'images with problems: {image_paths_with_errors}')

6748it [00:15, 436.32it/s]

Max width: 4800 image_path: ../../datasets/zalando-woman-outfit-dataset-raw/woman_outfits/9SY0L4DkQJK/4.jpg
Max height: 6912 image_path: ../../datasets/zalando-woman-outfit-dataset-raw/woman_outfits/9SY0L4DkQJK/4.jpg
images with problems: []


# Resize images

In [53]:

"""resize images to 256x256 pixels, keep the original aspect ratio of image and fill the void space with white"""


def resize_images(image_path, output_dir, size):
    # Load the image
    image = cv2.imread(image_path)

    # Check if image is loaded successfully
    if image is None:
        print(f"Error: Unable to load image from {image_path}")
        return

    # Get original dimensions
    height, width, _ = image.shape
    # Calculate aspect ratio
    aspect_ratio = width / height

    # Determine new dimensions based on size and aspect ratio
    if width > height:
        new_width = size
        new_height = int(size / aspect_ratio)
    else:
        new_height = size
        new_width = int(size * aspect_ratio)

    # Resize the image while maintaining aspect ratio
    resized_image = cv2.resize(image, (new_width, new_height))
    # Create a white canvas of size x size
    canvas = np.ones((size, size, 3), dtype=np.uint8) * 255
    # Calculate where to place the resized image on the canvas
    x_offset = (size - new_width) // 2
    y_offset = (size - new_height) // 2
    # Paste the resized image onto the canvas
    canvas[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_image
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    # Get the image filename from the original path
    filename = os.path.basename(image_path)
    # Save image to output path
    cv2.imwrite(os.path.join(output_dir, filename), canvas)


# Apply the resizing function to each row in the DataFrame
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    set_id = row['id']
    clothingItemIndex = row['clothingItemIndex']
    output_dir = f"{output_dir_base}/{set_id}"
    input_path = f"{images_dir}/{row['image_path']}"  # Adjust the input path
    resize_images(input_path, output_dir, size)

100%|██████████| 24551/24551 [10:06<00:00, 40.51it/s]


# Check if all Images are present in the output folder

In [56]:
"""check if all images from the dataframe are saved to the correct location in the new file system for resized images"""

missing_count = 0
missing_files = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    set_id = row['id']
    clothingItemIndex = row['clothingItemIndex']
    output_dir = f"{output_dir_base}/{set_id}"

    # Check if the resized image file exists
    if not os.path.exists(f'{output_dir}/{clothingItemIndex}.jpg'):
        missing_count += 1
        missing_files.append(output_dir)  # Store the path of missing file
        print(f"Resized image file is missing for {output_dir}")

# Print the total count of missing files
print(f"Total missing files: {missing_count}")

100%|██████████| 24551/24551 [00:01<00:00, 13603.04it/s]

Total missing files: 0
